In [17]:
import pandas as pd

file_path_1 = "/home/alex/dev/AiLearning/DataSetStore/twitter_messages/test.csv"

df1 = pd.read_csv(file_path_1)

file_path_2 = "/home/alex/dev/AiLearning/DataSetStore/twitter_messages/train.csv"

df2 = pd.read_csv(file_path_2)



In [18]:
import re
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
class MessageCleaner:
    
    def __init__(self,message=None):
        self._message= message
        self.tokens=[]
        # self.reg_map = {"removing_users":"@[\w]*",
        #                 "punctuation":'[%s]',#'\\[^\w\s]',
        #                 "special_symbols":"\\[^a-zA-Z0-9]",
        #                 "numbers_to_spaces":"\\[^a-zA-Z]" }

    def set_message(self,msg):
        self._message = msg;
        return self

    def get_message(self):
        return self._message

    def remove_users(self,replace=" "):
        xtr = re.sub(pattern="@[\w]*",repl=replace,string=self._message)
        #print ("USER",self._message,xtr)
        self._message = xtr
        return self
    
    def remove_punctuation(self,replace=" "):
        xtr = re.sub(pattern=r'[^\w\s]',repl=replace,string=self._message)
        #print ("PUNCT",self._message,xtr)
        self._message = xtr
        return self

    def remove_special_sym(self,replace=" "):
        xtr = re.sub(pattern=r'[^a-zA-Z0-9]',repl=replace,string=self._message)
        #print ("SS",self._message,xtr)
        self._message = xtr
        return self
    
    def remove_numbers(self,replace=" "):
        xtr = re.sub(pattern=r'[^a-zA-Z]',repl=replace,string=self._message)
        #print ("NN",self._message,xtr)
        self._message = xtr
        return self

    # def replace_by_regs (self,replace_map=None):
    #     for reg_key in self.reg_map:
    #         regexp = self.reg_map[reg_key]
    #         if replace_map != None and reg_key in replace_map:
    #             replace = replace_map[reg_key]
    #         else :
    #             replace = " "
    #         xtr = re.sub(pattern=regexp,repl=replace,string=self._message)
    #         print (regexp,self._message,xtr)
    #         self._message = xtr
    #     return self

    def to_lower (self ):
        self._message = self._message.lower();
        return self;

    def replace_by_dicts (self,dictionary: map):
        for key in dictionary:
            self._message = self._message.replace(key,dictionary[key])
            #re.sub(pattern=key,repl=dictionary[key],string=self._message)
        return self

    def escape_single_symbol_words (self):
        self._message=" ".join([word for word in self._message.split() if len(word)>1])
        return self

    def nltk_word_tokenize(self):
        #print ("1",self._message)
        self.tokens=tknz.word_tokenize(self._message)
        #print ("2",self.tokens)
        return self

    def nltk_word_punc_tokenize(self):
        self.tokens = tknz.wordpunct_tokenize(self._message)
        return self

    def nltk_tok_tok_tokenizer(self):
        self.tokens = tknz.ToktokTokenizer().tokenize(self._message)
        return self
        
    def nltk_tweet_tokenizer(self):
        self.tokens = tknz.TweetTokenizer().tokenize(self._message)
        return self

    def nltk_with_regexp_tokenizer (self,regexp):
        self.tokens = tknz.RegexpTokenizer(regexp).tokenize(self._message)
        return self

    def nltk_sentence_tokenizer (self):
        self.tokens = nltk.sent_tokenize(self._message)
        return self

    def remove_stopwords_from_tokens (self,lang=None,is_new=True):
        if lang is None:
            sw = set(stopwords.words("english"))
        else :
            sw = set(stopwords.words(lang))
        tks = [token for token in self.tokens if token not in sw]
        if is_new:
            self.tokens_without_stops = tks
        else :
            self.tokens = tks
        return self

    def stemme_tokens (self,stemmer=None,is_new=True):
        if stemmer is None:
            stemmer = PorterStemmer()
        xtr = [stemmer.stem(token) for token in self.tokens]
        if is_new :
            self.stem_tokens = xtr
        else :
            self.tokens = xtr
        return self

    def lematize_tokens (self,lematizer=None,is_new=True):
        if lematizer is None:
            lematizer = WordNetLemmatizer()
        xtr = [lematizer.lemmatize(token) for token in self.tokens]
        if is_new:
            self.lem_tokens = xtr
        else :
            self.tokens = xtr
        return self
            
    


In [15]:
import numpy as np
messages = []
df1['tweet'].apply(lambda m: messages.append(m))
df2['tweet'].apply(lambda m: messages.append(m))

message_cleaner = MessageCleaner()

new_messages=[]
tokens=[]
tokens_sw=[]
tokens_stm=[]
tokens_lm=[]

for message in messages:
    message_cleaner = MessageCleaner().set_message(message)
    #message_cleaner.replace_by_regs()
    message_cleaner.remove_users()
    message_cleaner.remove_punctuation()
    message_cleaner.remove_special_sym()
    message_cleaner.remove_numbers()
    message_cleaner.to_lower()
    message_cleaner.replace_by_dicts(dictionary={"can't":"can not","ain't": "are not"})
    message_cleaner.replace_by_dicts(dictionary={":)":"happy"})
    message_cleaner.escape_single_symbol_words()
    message_cleaner.nltk_word_tokenize()
    message_cleaner.remove_stopwords_from_tokens()
    message_cleaner.stemme_tokens().lematize_tokens()
    
    new_messages.append(message_cleaner.get_message())
    tokens.append(message_cleaner.tokens)
    tokens_sw.append(message_cleaner.tokens_without_stops)
    tokens_stm.append(message_cleaner.stem_tokens)
    tokens_lm.append(message_cleaner.lem_tokens)
    


In [16]:
dict = {'tweet': messages,'tweet_token': tokens,'tweet_token_filtered': tokens_sw, 'tweet_stemmed': tokens_stm, 'tweet_lemmatized': tokens_lm} 
df = pd.DataFrame(dict)
df.to_pickle("xxxx.pickle")
df

,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,#studiolife #aislife #requires #passion #dedic...,"[studiolife, aislife, requires, passion, dedic...","[studiolife, aislife, requires, passion, dedic...","[studiolif, aislif, requir, passion, dedic, wi...","[studiolife, aislife, requires, passion, dedic..."
1,@user #white #supremacists want everyone to s...,"[white, supremacists, want, everyone, to, see,...","[white, supremacists, want, everyone, see, new...","[white, supremacist, want, everyon, to, see, t...","[white, supremacist, want, everyone, to, see, ..."
2,safe ways to heal your #acne!! #altwaystohe...,"[safe, ways, to, heal, your, acne, altwaystohe...","[safe, ways, heal, acne, altwaystoheal, health...","[safe, way, to, heal, your, acn, altwaystoh, h...","[safe, way, to, heal, your, acne, altwaystohea..."
3,is the hp and the cursed child book up for res...,"[is, the, hp, and, the, cursed, child, book, u...","[hp, cursed, child, book, reservations, alread...","[is, the, hp, and, the, curs, child, book, up,...","[is, the, hp, and, the, cursed, child, book, u..."
4,"3rd #bihday to my amazing, hilarious #nephew...","[rd, bihday, to, my, amazing, hilarious, nephe...","[rd, bihday, amazing, hilarious, nephew, eli, ...","[rd, bihday, to, my, amaz, hilari, nephew, eli...","[rd, bihday, to, my, amazing, hilarious, nephe..."
...,...,...,...,...,...
49154,ate @user isz that youuu?ðððððð...,"[ate, isz, that, youuu]","[ate, isz, youuu]","[ate, isz, that, youuu]","[ate, isz, that, youuu]"
49155,to see nina turner on the airwaves trying to...,"[to, see, nina, turner, on, the, airwaves, try...","[see, nina, turner, airwaves, trying, wrap, ma...","[to, see, nina, turner, on, the, airwav, tri, ...","[to, see, nina, turner, on, the, airwave, tryi..."
49156,listening to sad songs on a monday morning otw...,"[listening, to, sad, songs, on, monday, mornin...","[listening, sad, songs, monday, morning, otw, ...","[listen, to, sad, song, on, monday, morn, otw,...","[listening, to, sad, song, on, monday, morning..."
49157,"@user #sikh #temple vandalised in in #calgary,...","[sikh, temple, vandalised, in, in, calgary, ws...","[sikh, temple, vandalised, calgary, wso, conde...","[sikh, templ, vandalis, in, in, calgari, wso, ...","[sikh, temple, vandalised, in, in, calgary, ws..."
